<a href="https://colab.research.google.com/github/shreyasacharya1/ai_lab/blob/main/resolution_first_order.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print("Shreyas.R.Achar 1BM23CS320")
from copy import deepcopy

# ------------------------------
# Utility functions
# ------------------------------
def substitute(clause, subs):
    """Apply substitutions to a clause."""
    new_clause = []
    for literal in clause:
        pred, args, neg = literal
        new_args = []
        for a in args:
            if a in subs:
                new_args.append(subs[a])
            else:
                new_args.append(a)
        new_clause.append((pred, new_args, neg))
    return new_clause

def unify(x, y, subs=None):
    """Unify two literals/terms."""
    if subs is None:
        subs = {}
    if x == y:
        return subs
    if isinstance(x, str) and x[0].islower():
        return unify_var(x, y, subs)
    elif isinstance(y, str) and y[0].islower():
        return unify_var(y, x, subs)
    elif isinstance(x, tuple) and isinstance(y, tuple):
        if x[0] != y[0] or len(x[1]) != len(y[1]):
            return None
        for a, b in zip(x[1], y[1]):
            subs = unify(a, b, subs)
            if subs is None:
                return None
        return subs
    else:
        return None

def unify_var(var, x, subs):
    if var in subs:
        return unify(subs[var], x, subs)
    elif x in subs:
        return unify(var, subs[x], subs)
    elif occur_check(var, x, subs):
        return None
    else:
        subs[var] = x
        return subs

def occur_check(var, x, subs):
    if var == x:
        return True
    elif isinstance(x, str) and x in subs:
        return occur_check(var, subs[x], subs)
    elif isinstance(x, tuple):
        return any(occur_check(var, arg, subs) for arg in x[1])
    return False


# ------------------------------
# Resolution core
# ------------------------------
def resolve(clause1, clause2):
    """Try all pairs of literals to resolve."""
    resolvents = []
    for lit1 in clause1:
        for lit2 in clause2:
            if lit1[0] == lit2[0] and lit1[2] != lit2[2]:  # same predicate, opposite polarity
                subs = unify(tuple(lit1[:2]), tuple(lit2[:2]))
                if subs is not None:
                    new_c1 = substitute(clause1, subs)
                    new_c2 = substitute(clause2, subs)
                    new_clause = [l for l in new_c1 if l != lit1] + [l for l in new_c2 if l != lit2]
                    # remove duplicates
                    new_clause = [x for i, x in enumerate(new_clause) if x not in new_clause[:i]]
                    resolvents.append(new_clause)
    return resolvents


def pl_resolution(kb, query):
    clauses = deepcopy(kb)
    # add negated query
    neg_query = [(query[0], query[1], not query[2])]
    clauses.append(neg_query)

    print("Initial clauses:")
    for i, c in enumerate(clauses):
        print(f"C{i+1}: {pretty_clause(c)}")
    print("\nResolution steps:\n")

    new = []
    while True:
        n = len(clauses)
        pairs = [(clauses[i], clauses[j]) for i in range(n) for j in range(i + 1, n)]
        for (ci, cj) in pairs:
            resolvents = resolve(ci, cj)
            for r in resolvents:
                if not r:
                    print("Derived empty clause {} -> CONTRADICTION.".format(pretty_clause(r)))
                    return True
                if r not in clauses and r not in new:
                    new.append(r)
                    print(f"Resolved {pretty_clause(ci)} and {pretty_clause(cj)} -> {pretty_clause(r)}")
        if all(r in clauses for r in new):
            return False
        clauses.extend(new)


# ------------------------------
# Pretty printer
# ------------------------------
def pretty_clause(clause):
    if not clause:
        return "{}"
    out = []
    for pred, args, neg in clause:
        s = f"{pred}({', '.join(args)})"
        if neg:
            s = "¬" + s
        out.append(s)
    return " ∨ ".join(out)


# ------------------------------
# Example Knowledge Base
# ------------------------------
if __name__ == "__main__":
    # KB Clauses:
    # 1. ¬parent(x,y) ∨ ancestor(x,y)
    # 2. ¬ancestor(x,y) ∨ ¬ancestor(y,z) ∨ ancestor(x,z)
    # 3. parent(tom,bob)
    # 4. parent(bob,alice)
    KB = [
        [('parent', ['x', 'y'], True), ('ancestor', ['x', 'y'], False)],
        [('ancestor', ['x', 'y'], True), ('ancestor', ['y', 'z'], True), ('ancestor', ['x', 'z'], False)],
        [('parent', ['tom', 'bob'], False)],
        [('parent', ['bob', 'alice'], False)]
    ]

    # Query: ancestor(tom, alice)
    QUERY = ('ancestor', ['tom', 'alice'], False)

    result = pl_resolution(KB, QUERY)
    print("\nResult:", "PROVED ✅" if result else "NOT PROVABLE ❌")


Streaming output truncated to the last 5000 lines.
Resolved ¬ancestor(x, y) ∨ ¬ancestor(y, z) ∨ ancestor(x, z) and ¬ancestor(z, y) ∨ ¬ancestor(y, y) ∨ ancestor(z, y) ∨ ¬ancestor(y, z) ∨ ancestor(z, z) ∨ parent(y, z) ∨ ancestor(y, z) -> ¬ancestor(x, z) ∨ ¬ancestor(z, z) ∨ ancestor(x, z) ∨ ancestor(z, z) ∨ parent(z, z)
Resolved ¬ancestor(x, y) ∨ ¬ancestor(y, z) ∨ ancestor(x, z) and ¬ancestor(z, y) ∨ ¬ancestor(y, y) ∨ ancestor(z, y) ∨ ¬ancestor(y, z) ∨ ancestor(z, z) ∨ parent(y, z) ∨ ancestor(y, z) -> ¬ancestor(x, y) ∨ ancestor(x, z) ∨ ¬ancestor(z, y) ∨ ¬ancestor(y, y) ∨ ancestor(z, y) ∨ ¬ancestor(y, z) ∨ ancestor(z, z) ∨ parent(y, z)
Resolved ¬ancestor(x, y) ∨ ¬ancestor(y, z) ∨ ancestor(x, z) and ¬ancestor(z, y) ∨ ¬ancestor(y, y) ∨ ancestor(z, y) ∨ ¬ancestor(y, z) ∨ ancestor(z, z) ∨ parent(y, z) ∨ ancestor(y, z) -> ¬ancestor(y, y) ∨ ¬ancestor(y, z) ∨ ancestor(y, z) ∨ ¬ancestor(z, y) ∨ ancestor(z, y) ∨ ancestor(z, z) ∨ parent(y, z)
Resolved ¬ancestor(x, y) ∨ ¬ancestor(y, z) ∨ ancestor(x, 